# 代码测试
Pulp包需要第三方下载
-pip install pulp

In [3]:
#算法部分，使用pulp内置cdc求解器
import pulp
import numpy as np
import pandas as pd
class ILP():
    def __init__(self,c,T,S):
        #输入说明
        #T:饲料厂产量
        #S:养殖场需求
        #l：距离矩阵
        self.T = T
        self.S = S
        self.row = len(self.T)
        self.col = len(self.S)
        self.c = np.array(c).reshape(self.row,self.col)
        
    def solve(self):
        row = len(self.T) #饲料厂个数
        col = len(self.S) #养殖场个数
        
        prob = pulp.LpProblem('运输配给优化', sense=pulp.LpMinimize)
        
        var = [[pulp.LpVariable(f'x{i}_{j}', cat=pulp.LpBinary) for j in range(col)] for i in range(row)]

        flatten = lambda x: [y for l in x for y in flatten(l)] if type(x) is list else [x]

        prob += pulp.lpDot(flatten(var), self.c.flatten())

        for i in range(row):
            prob += (pulp.lpDot(pulp.lpSum(var[i]),self.S) <= self.T[i])

        for j in range(col):
            prob += (pulp.lpDot([var[i][j] for i in range(row)], [1]*row) == 1)
        pulp.LpSolverDefault.msg = 1
        prob.solve()
        self.opt_val = round(pulp.value(prob.objective),2)
        self.opt_x = pd.DataFrame([[pulp.value(var[i][j]) for j in range(col)] for i in range(row)])
        #输出说明：
        #opt_val为最低运输距离
        #opt_x为最优运输方式，格式为row（饲料厂数）*col（养殖场数）的0-1矩阵。x[i,j]代表第i间饲料厂是否给第j间养殖场配送饲料
        
#部分数据测试及结果

# 算法测试部分

In [4]:
#加载使用的package以及读取相关文件
#测试2020年1月份数据
import pulp
import numpy as np
import pandas as pd
月份 = '2020-01'
距离矩阵 = pd.read_excel('Data1.xlsx')
养殖场 = pd.read_csv('Data2.csv',encoding = 'gbk')
饲料厂 = pd.read_excel('Data3.xlsx')
距离矩阵 = 距离矩阵[['FieldName','FeedName','FDistance']]
距离矩阵 = 距离矩阵.rename(columns = {'FieldName':'养殖场','FeedName' : '饲料厂', 'FDistance' : '距离'})
养殖场 = 养殖场[养殖场.月份 == 月份][['场区名称','日需求量']]
养殖场 = 养殖场.rename(columns = {'场区名称':'养殖场'})
饲料厂 =饲料厂[['FProjectName','FScaleAmount']]
饲料厂 = 饲料厂.rename(columns = {'FProjectName':'饲料厂','FScaleAmount':'产能'}).dropna()

In [7]:
#查看缺失值
养殖场.日需求量.isnull().sum()
饲料厂.产能.isnull().sum()

0

In [9]:
#获得用来制作矩阵的pivot table
data = pd.merge(距离矩阵,养殖场,how = 'inner',on = '养殖场')
data = pd.merge(data,饲料厂,how = 'inner', on = '饲料厂')

In [33]:
df1 = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '距离')
print(df1.index[np.where(np.isnan(df1))[0]], df1.columns[np.where(np.isnan(df1))[1]])
#缺失颍上2场到万荣饲料厂1厂 的距离。这个数字在测试阶段我选择随机填写为500公里
#在这里也可以选择删掉所有和'颍上2场'或与'万荣饲料厂1厂'有关的数据
#如果爬虫获得的数据完整这里应该没有缺失值

Index(['颍上2场'], dtype='object', name='养殖场') Index(['万荣饲料厂1厂'], dtype='object', name='饲料厂')


如果距离不完整，算法程序会报错。
这里因为缺少了颍上2场到万荣饲料厂1厂的距离，制作的pivot table里关于颍上2场的日需求量需要手动填写

In [48]:
df2 = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '日需求量')
print(df2.index[np.where(np.isnan(df2))[0]])
#print(df2.iloc['颍上2场']

Index(['颍上2场'], dtype='object', name='养殖场')


In [49]:
df3 = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '产能')
print(df3.columns[np.where(np.isnan(df3))[1]])

Index(['万荣饲料厂1厂'], dtype='object', name='饲料厂')


In [63]:
#获得距离矩阵，填补缺失值
l = np.array(df1)
S = np.array(df2.iloc[:,0])
T = np.array(df3.iloc[0,:])*10000/330
#pd.pivot_table(df,index=[u'对手',u'主客场'])

In [64]:
#手动填写错漏的数据
养殖场[养殖场['养殖场'] == '颍上2场']
S[350] = 50.67
l[350,0] = 500000

In [65]:
import time
start = time.clock()
a = ILP(l,T,S)
a.solve()
print('最低运输距离为：',a.opt_val)
print('最优供给关系为：','\n',a.opt_x)
elapsed = (time.clock() - start)
print("Time used:",elapsed)
#运算时间大概不到6秒， 结果是117*372的矩阵，0代表该饲料厂和养殖场没有供给关系，1代表由该饲料厂供给此养殖场

E:\jupyter\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  


最低运输距离为： 9335872.0
最优供给关系为： 
      0    1    2    3    4    5    6    7    8    9    ...  362  363  364  \
0    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3    0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
5    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
6    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
7    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
8    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
9    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
10   0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
11   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0

E:\jupyter\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [66]:
#输出表格
S_name = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '日需求量').index
T_name = pd.pivot_table(data,index=['养殖场'],columns = ['饲料厂'],values = '产能').columns
a.opt_x.columns = S_name
a.opt_x.index = T_name
a.opt_x.to_csv('result.csv')
result = a.opt_x

In [ ]:
# SQL部分，未完成
'''import pandas as pd
import pymysql
# sql 命令
sql_cmd1 = "SELECT * FROM table"
sql_cmd1 = "SELECT * FROM table"

con = pymysql.connect(host=localhost, user=username, password=password, database=dbname, charset='utf8', use_unicode=True)
#df1:养殖场及其需求，注意统一需求的时间，日需求或是月需求
df1 = pd.read_sql(sql_cmd1, con)
#df2 :饲料厂及其产能
df2 = pd.read_sql(sql_cmd2, con)
#data: 爬虫出来的距离矩阵
data = pd.read_csv('Data1.csv',encoding = 'gbk',sep= '\t',header = None).iloc[:,[1,5,9]]
#测试月份是2020年1月
data= pd.merge(data,data_1[data_1['月份']=='2020-01'],how = 'inner', on = '养殖场')
data =pd.merge(data,data_2,how = 'inner', on = '饲料厂')'''